In [25]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as no

In [26]:
##load the trained model ann,scaler pickle ,onehot

model = load_model('model.h5')
##load the encoder and scaler
with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo = pickle.load(file)

with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_geo = pickle.load(file)

with open('sclaer.pkl','rb') as file:
    sclaer_geo = pickle.load(file)

In [27]:
##example input
input_data = {
    'CreditScore' : 600,
    'Geography' : 'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance' : 60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [28]:
input_data

{'CreditScore': 600,
 'Geography': 'France',
 'Gender': 'Male',
 'Age': 40,
 'Tenure': 3,
 'Balance': 60000,
 'NumOfProducts': 2,
 'HasCrCard': 1,
 'IsActiveMember': 1,
 'EstimatedSalary': 50000}

In [29]:
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(
    geo_encoded,
    columns=onehot_encoder_geo.get_feature_names_out(['Geography'])
)

c:\Users\Karthik\anaconda3\envs\myenv\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [30]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [31]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [32]:
#encode categorical variables

input_df['Gender'] = label_encoder_geo.transform(input_df['Gender'])
input_df['Gender']

0    1
Name: Gender, dtype: int64

In [35]:
#concat with Onehot encoded 
input_df = pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [36]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [37]:
##scaling the input data
inpput_sclaed = sclaer_geo.transform(input_df)


In [38]:
inpput_sclaed

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [39]:
##predict churn
prediciton = model.predict(inpput_sclaed)
prediciton

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step


array([[0.02385981]], dtype=float32)

In [41]:
prediciton_proba = prediciton[0][0]

In [42]:
prediciton_proba

np.float32(0.02385981)

In [43]:
if prediciton_proba > 0.5:
    print("the customer is likely to churn")
else:
    print("the customer is not likely to churn")

the customer is not likely to churn
